In [1]:
import torch

torch.cuda.set_device(0)
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, RobertaTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import torch 
from peft import LoraConfig, get_peft_model
import transformers 
import os 
# os.environ["TRANSFORMERS_CACHE"] = "/workspace/yo"
from datasets import Dataset

import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import torch.nn as nn
import torch
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict
# import wandb
# os.environ["WANDB_PROJECT"] = "merged-roberta" # name your W&B project 
# os.environ["WANDB_LOG_MODEL"] = "checkpoint" # log all model checkpoints
# wandb.login()

In [2]:
def calc_weights(total, nclass):
    weight = total/(nclass*6)
    return weight

c0 = calc_weights(133755, 65351)
c1 = calc_weights(133755, 14339)
c2 = calc_weights(133755, 13678)
c3 = calc_weights(133755, 14343)
c4 = calc_weights(133755, 11998)
c5 = calc_weights(133755, 14046)

print(c0, c1, c2, c3, c4, c5)

import torch
import torch.nn as nn
torch.cuda.set_device(0)
device = torch.device("cuda")


def wce_loss(logits, labels):
    class_weights = torch.tensor([c0, c1, c2, c3, c4, c5]).to(device)
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    return loss_func(logits, labels).to(device)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
#         labels = labels.to(device)
        outputs = model(**inputs)
#         outputs = outputs.to(device)
        logits = outputs.logits.view(-1, self.model.config.num_labels)
#         logits = logits.to(device)
        loss = wce_loss(logits, labels)
#         loss = loss.to(device)
        return (loss, outputs) if return_outputs else loss

0.34111949319826784 1.5546760583025316 1.6298069893259248 1.5542424876246252 1.8580180030005 1.5871066495799515


In [4]:
def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs['tokenizer'](examples["text"], truncation=True, max_length=512, padding=True, return_tensors="pt")


def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results

In [6]:
id2label = {0: 'human', 1: 'chatGPT', 2: 'cohere', 3: 'davinci', 4: 'bloomz', 5: 'dolly'}
label2id = {'human': 0, 'chatGPT': 1,'cohere': 2, 'davinci': 3, 'bloomz': 4, 'dolly': 5}
test_path =  "/workspace/merged_dataset/subtaskB_test.jsonl"
test_df = pd.read_json(test_path, lines=True)

In [8]:
def get_data(train_path, test_path, val_path,random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)
    val_df = pd.read_json(val_path, lines=True)

    return train_df, val_df, test_df

In [9]:
random_seed = 0
train_path =  "/workspace/merged_dataset/subtaskB_train_merged.jsonl" # For example 'subtaskA_train_multilingual.jsonl'
test_path =  "/workspace/merged_dataset/subtaskB_test.jsonl" # For example 'subtaskA_test_multilingual.jsonl'
val_path = "/workspace/merged_dataset/subtaskB_dev.jsonl"

model =  "roberta-large"
checkpoints_path = f"/workspace/Models/{model}/"
model_name = model

set_seed(random_seed)

train_df, valid_df, test_df = get_data(train_path, test_path, val_path, random_seed)

# pandas dataframe to huggingface Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)


# get tokenizer and model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(
   checkpoints_path, num_labels=len(label2id), id2label=id2label, label2id=label2id#, quantization_config=bnb_config, device_map={"":0}    # put your model here
)


tokenizer = AutoTokenizer.from_pretrained(checkpoints_path)    # put your model here


# tokenize data for train/valid
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/tmp/ipykernel_4741/1438207522.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  train_df = pd.read_json(train_path, lines=True)


ValueError: Expected object or value

In [7]:
model_path = "/workspace/Models/roberta-large/checkpoint-33440"

# load tokenizer from saved model
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load best model
model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

#     model.add_adapter(config, "lora")

# model.load_state_dict(torch.load(checkpoints_path+'/best'+'/model.pt'))

test_dataset = Dataset.from_pandas(test_df)

tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# create Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
# get logits from predictions and evaluate results using classification report
predictions = trainer.predict(tokenized_test_dataset)
prob_pred = softmax(predictions.predictions, axis=-1)
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("bstrai/classification_report")
results = metric.compute(predictions=preds, references=predictions.label_ids)

results

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'0': {'precision': 0.9926127527216174,
  'recall': 0.851,
  'f1-score': 0.916367552045944,
  'support': 3000.0},
 '1': {'precision': 0.5836901518100428,
  'recall': 0.9996666666666667,
  'f1-score': 0.7370361268124846,
  'support': 3000.0},
 '2': {'precision': 0.9927884615384616,
  'recall': 0.2753333333333333,
  'f1-score': 0.4311064718162839,
  'support': 3000.0},
 '3': {'precision': 0.6800119510008963,
  'recall': 0.7586666666666667,
  'f1-score': 0.7171892232550812,
  'support': 3000.0},
 '4': {'precision': 0.9021968101113451,
  'recall': 0.9993333333333333,
  'f1-score': 0.9482840423849439,
  'support': 3000.0},
 '5': {'precision': 0.9705882352941176,
  'recall': 0.902,
  'f1-score': 0.93503800967519,
  'support': 3000.0},
 'accuracy': 0.7976666666666666,
 'macro avg': {'precision': 0.8536480604127469,
  'recall': 0.7976666666666666,
  'f1-score': 0.7808369043316546,
  'support': 18000.0},
 'weighted avg': {'precision': 0.8536480604127468,
  'recall': 0.7976666666666666,
  'f1-sc

In [20]:
model_path = "/workspace/Models/roberta-large/checkpoint-41800"

# load tokenizer from saved model
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load best model
model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

#     model.add_adapter(config, "lora")

# model.load_state_dict(torch.load(checkpoints_path+'/best'+'/model.pt'))

test_dataset = Dataset.from_pandas(test_df)

tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# create Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
# get logits from predictions and evaluate results using classification report
predictions = trainer.predict(tokenized_test_dataset)
prob_pred = softmax(predictions.predictions, axis=-1)
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("bstrai/classification_report")
results = metric.compute(predictions=preds, references=predictions.label_ids)

results

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'0': {'precision': 0.9944710652414301,
  'recall': 0.8993333333333333,
  'f1-score': 0.944512515315946,
  'support': 3000.0},
 '1': {'precision': 0.6525239338555265,
  'recall': 0.9996666666666667,
  'f1-score': 0.7896261190100052,
  'support': 3000.0},
 '2': {'precision': 0.9955290611028316,
  'recall': 0.44533333333333336,
  'f1-score': 0.6153846153846154,
  'support': 3000.0},
 '3': {'precision': 0.7530364372469636,
  'recall': 0.806,
  'f1-score': 0.7786185799388182,
  'support': 3000.0},
 '4': {'precision': 0.9362898188632105,
  'recall': 0.9993333333333333,
  'f1-score': 0.9667849080941632,
  'support': 3000.0},
 '5': {'precision': 0.978882833787466,
  'recall': 0.958,
  'f1-score': 0.9683288409703504,
  'support': 3000.0},
 'accuracy': 0.8512777777777778,
 'macro avg': {'precision': 0.8851221916829047,
  'recall': 0.8512777777777778,
  'f1-score': 0.8438759297856498,
  'support': 18000.0},
 'weighted avg': {'precision': 0.8851221916829047,
  'recall': 0.8512777777777778,
  'f1-

In [19]:
!rm -rf /workspace/Models/roberta-large/checkpoint-25080

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
